In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import cv2
import numpy as np
import os.path
import requests
import statistics
from enum import IntEnum
from PIL import Image, ImageDraw

Some information about the games from the https://www.harmmade.com/vectorracer/#

* Map contains grid with the step equal to 15
* Tracks must be exactly 794 pixels wide and at least 599 pixels high
* The grid spacing is 15 pixels; the start point is used as an anchor point for the grid

In [ ]:
def download_image(img):
    if os.path.exists(img):
        return
    data = requests.get('https://www.harmmade.com/vectorracer/tracks/{}'.format(img)).content
    with open(img, 'wb') as handler:
        handler.write(data)
      
class Track:
    def __init__(self, image):
        self._orig = image
        self._classify_colors()
        self._start = self._find_start()
        self._canvas = self._orig.copy()
        self._track = self._fill_track()
        self._grid_step = 15

    def save_canvas(self, name):
        self._canvas.save(name)
    
    def _classify_colors(self):
        self.TRACK = 20
        self.BOUNDARY = None
        self.CH1 = []
        self.CH2 = []
        self.CH3 = []
        self.CH4 = []
        self.DIR = []
        self.START = []
        self.FINISH = None
        self.EMPTY = None
        palette = self._orig.getpalette()
        for _, idx in self._orig.getcolors():
            cl = palette[3*idx], palette[3*idx+1], palette[3*idx+2]
            match cl:
                case (60, 120, 240):
                    self.BOUNDARY = idx
                case (104, 255, 104) | ( 80, 236, 98):
                    self.CH1.append(idx)
                case (255, 104, 104) | (230,  86, 98):
                    self.CH2.append(idx)
                case (255, 104, 255) | (230,  86, 248):
                    self.CH3.append(idx)
                case (255, 179, 104) | (230, 161, 98):
                    self.CH4.append(idx)
                case (195, 195, 195) | (149, 160, 183):
                    self.DIR.append(idx)
                case (155, 155, 155) | ( 36,  72, 145) | (118, 127, 145):
                    self.START.append(idx)
                case (255, 255, 255):
                    self.EMPTY = idx
                case ( 0, 0, 0):
                    self.FINISH = idx
                case (195, 210, 240):
                    self.GRID = idx
    
    def _find_start(self):
        xs = []
        ys = []
        for x in range(self._orig.width):
            for y in range(self._orig.height):
                v = self._orig.getpixel((x,y))
                if v in self.START:
                    xs.append(x)
                    ys.append(y)
        return statistics.median(xs), statistics.median(ys)

    def _fill_track(self):
        px = np.array(self._orig)
        px[px != self.BOUNDARY] = self.EMPTY
        cv2.floodFill(px, None, self._start, self.TRACK, 0, 1)
        px[px != self.TRACK] = self.EMPTY
        rv = Image.fromarray(px, mode='P')
        rv.putpalette(self._orig.getpalette())
        return rv
       
    def pt(self, pos):
        return (self._start[0] + self._grid_step * pos[0], self._start[1] - self._grid_step * pos[1])
    
    def is_valid(self, pos):
        p = self.pt(pos)
        return p[0] >= 0 and p[1] >= 0 and p[0] < self._orig.width and p[1] < self._orig.height
      
    def is_on_track(self, pos):
        p = self.pt(pos)
        return self.is_valid(pos) and self._track.getpixel(p) == self.TRACK
    
    def is_on_finish(self, pos):
        p = self.pt(pos)
        return self.is_valid(pos) and self._orig.getpixel(p) == self.FINISH
    
    def _intersects_finish_back(self, pos0, pos1):
        return pos0[0] >= 0 and pos1[0] < 0 and pos1[1] < 10
    
    def is_on_track_and_finishes(self, pos0, pos1):
        if self._intersects_finish_back(pos0, pos1):
            return False
        if not self.is_on_track(pos0):
            return False
        for p in np.linspace(pos0, pos1, 10):
            if self.is_on_finish(p):
                return True
            if not self.is_on_track(p):
                return False
        return self.is_on_finish(pos1)
    
    def find_finish(self, pos0, pos1):
        for p in np.linspace(pos0, pos1, 100):
            if self.is_on_finish(p):
                return p
        return None        

    def find_finish_param(self, pos0, pos1):
        p = self.find_finish(pos0, pos1)
        if p is None:
            return None
        num = (pos1[0]-pos0[0])*(p[0]-pos0[0]) + (pos1[1]-pos0[1])*(p[1]-pos0[1])
        denom = (pos1[0]-pos0[0])*(pos1[0]-pos0[0]) + (pos1[1]-pos0[1])*(pos1[1]-pos0[1])
        return num / denom
        
    def is_on_track_segment(self, pos0, pos1):
        if self._intersects_finish_back(pos0, pos1):
            return False
        if not self.is_on_track(pos0):
            return False
        for p in np.linspace(pos0, pos1, 10):
            if self.is_on_finish(p):
                return True
            if not self.is_on_track(p):
                return False
        return self.is_on_track(pos1)
    
    def draw(self, pos):
        if self.is_on_track(pos):
            d = 3
            fill = 6
        else:
            d = 10
            fill = 3
        img = ImageDraw.Draw(self._canvas)
        (px, py) = self.pt(pos)
        img.ellipse([(px-d, py-d), (px+d, py+d)], fill=fill, outline=30)
        
    def draw_segment(self, pos0, pos1):
        if self.is_on_track_segment(pos0, pos1):
            fill = 77
            width = 0
        else:
            fill = 3
            width = 3
        p0 = self.pt(pos0)
        p1 = self.pt(pos1)
        img = ImageDraw.Draw(self._canvas)
        img.line([p0, p1], fill=fill, width=width)
        
    def draw_trajectory(self, pts):
        for i in range(1, len(pts)):
            self.draw_segment(pts[i-1], pts[i])
        for pt in pts:
            self.draw(pt)
            
    def neighbours(self, pos):
        rv = []
        for ddx in (-1, 0, 1):
            for ddy in (-1, 0, 1):
                (x, y, dx, dy) = pos
                s = (x + dx + ddx, y + dy + ddy, dx + ddx, dy + ddy)
                if abs(s[2]) > 6 or abs(s[3]) > 6:
                    continue
                if not self.is_on_track_segment(pos, s):
                    continue
                rv.append(s)
        return rv
                
def get_dst_sqr(pos0, pos1):
    (x0, y0) = pos0
    (x1, y1) = pos1
    return (x1-x0)*(x1-x0)+(y1-y0)*(y1-y0)
    

In [ ]:
download_image('oval.png')
track = Track(Image.open('oval.png'))

In [ ]:
def bfs(tr):
    state = (0, 0, 0, 0)
    l = [state]
    idx = 0
    d = {state: 0}
    back = {}
    finishes = []
    found_distance = None
    while idx < len(l):
        el = l[idx]
        dist = d[el]
        idx = idx + 1
        if not found_distance is None and dist >= found_distance:
            break
        for s in track.neighbours(el):
            if s in d:
                continue
            back[s] = el
            d[s] = dist + 1
            if tr.is_on_track_and_finishes(el, s):
                finishes.append(s)
                found_distance = dist + 1
            else:
                l.append(s)
    return finishes, back

def trajectory(finish, back):
    tr = [finish]
    while tr[-1] in back:
        tr.append(back[tr[-1]])
    return list(reversed(tr))

def trajectory_length(track, tr):
    b = tr[-1]
    a = tr[-2]
    p = round(track.find_finish_param(a, b), 2)
    return len(tr) - 1 + p

def last_speed(tr):
    a = tr[-1]
    return a[2] * a[2] + a[3] * a[3]

In [ ]:
%%time

finishes, back = bfs(track)
trajectories = [trajectory(f, back) for f in finishes]
weigthed_trajectories = sorted([(trajectory_length(track, tr), last_speed(tr), tr) for tr in trajectories])
best = weigthed_trajectories[0][2]

In [ ]:
track.draw_trajectory(best)
track.save_canvas('oval-best.png')